In [1]:
# Original datasets from the first part of the project
! curl "https://api.mockaroo.com/api/dde01370?count=1000&key=11149690" > "customers.csv"
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "products.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
100  1537    0  1537    0     0    249      0 --:--:--  0:00:06 --:--:--   278
100  7013    0  7013    0     0   1086      0 --:--:--  0:00:06 --:--:--  1393
100 41272    0 41272    0     0   5371      0 --:--:--  0:00:07 --:--:--  9249
100 69905    0 69905    0     0   7944      0 --:--:--  0:00:08 --:--:-- 14219
100 69905    0 69905    0     0   7182      0 --:--

In [2]:
! curl "https://api.mockaroo.com/api/9bb4af90?count=200&key=12883420" > "Employees_Data.csv"
! curl "https://api.mockaroo.com/api/8edb9c10?count=1000&key=12883420" > "Orders_Data.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  6789    0  6789    0     0   3078      0 --:--:--  0:00:02 --:--:--  3092
100 19141    0 19141    0     0   7940      0 --:--:--  0:00:02 --:--:--  7975
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 33044    0 33044    0     0  13754      0 --:--:--  0:00:02 --:--:-- 13808
100 70325    0 70325    0     0  23318      0 --:--:--  0:00:03 --:--:-- 23394
100 70325    0 70325    0     0  23304      0 --:-

In [7]:
import sqlite3
import pandas as pd
import os

In [4]:
customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50),
    email VARCHAR(50), phone_number VARCHAR(50), address VARCHAR(50),
    city VARCHAR(50), country VARCHAR(50), postal_code VARCHAR(50), loyalty_points INT
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY, product_name TEXT, description TEXT, price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2), category VARCHAR(50), brand TEXT, stock_quantity INT,
    color VARCHAR(50), size VARCHAR(20), weight DECIMAL(5,2), dimensions TEXT,
    release_date DATE, rating DECIMAL(3,1), reviews_count INT, seller_name TEXT,
    seller_rating DECIMAL(3,1), seller_reviews_count INT, shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
"""

orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY, customer_id INT, product_id INT, quantity INT,
    unit_price DECIMAL(10,2), total_price DECIMAL(10,2), order_date DATE,
    shipping_address VARCHAR(255), payment_method VARCHAR(20), status VARCHAR(20),
    employee_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    job_title VARCHAR(50),
    department VARCHAR(50),
    hire_date DATE,
    salary INT,
    manager_id INT
);
"""

In [10]:
db_name = 'ecommerce_with_employees.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database {db_name}")

In [11]:
try:
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' is created.")

    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(employees_schema)
    cursor.execute(orders_schema)
    print("All Tables schemas created successfully!")

    csv_to_table_map = {'customers.csv': 'customers', 'Employees_Data.csv': 'employees', 'Orders_Data.csv': 'orders', 'products.csv': 'products'}
    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            df.to_sql(table_name, con=conn, if_exists='append', index=False)
            print(f"--> Data from '{csv_file}' is successfully loaded into '{table_name}'")
        else:
            print(f"Warning! {csv_file} not found")
except Exception as e:
    print(f"Error occured: {e}")
finally:
    conn.commit()
    conn.close()

Database 'ecommerce_with_employees.db' is created.
All Tables schemas created successfully!
--> Data from 'customers.csv' is successfully loaded into 'customers'
--> Data from 'Employees_Data.csv' is successfully loaded into 'employees'
--> Data from 'Orders_Data.csv' is successfully loaded into 'orders'
--> Data from 'products.csv' is successfully loaded into 'products'
